# 文中介绍SAGAN中的attention

- 来自于https://github.com/heykeetae/Self-Attention-GAN
- https://zhuanlan.zhihu.com/p/110130098

## 源码

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from spectral import SpectralNorm
import numpy as np

class Self_Attn(nn.Module):
    """ Self attention Layer"""
    def __init__(self,in_dim,activation):
        super(Self_Attn,self).__init__()
        self.chanel_in = in_dim
        self.activation = activation
        self.query_conv = nn.Conv2d(in_channels = in_dim , out_channels = in_dim//8 , kernel_size= 1)
        self.key_conv = nn.Conv2d(in_channels = in_dim , out_channels = in_dim//8 , kernel_size= 1)
        self.value_conv = nn.Conv2d(in_channels = in_dim , out_channels = in_dim , kernel_size= 1)
        self.gamma = nn.Parameter(torch.zeros(1))

        self.softmax  = nn.Softmax(dim=-1) #
    def forward(self,x):
        """
            inputs :
                x : input feature maps( B X C X W X H)
            returns :
                out : self attention value + input feature 
                attention: B X N X N (N is Width*Height)
        """
        m_batchsize,C,width ,height = x.size()
        
        # B X C X (N) permute(0, 2, 1)相当于对矩阵进行了转置
        proj_query  = self.query_conv(x).view(m_batchsize,-1,width*height).permute(0,2,1) 
        proj_key =  self.key_conv(x).view(m_batchsize,-1,width*height) # B X C x (*W*H)
        
        energy =  torch.bmm(proj_query,proj_key) # transpose check
        attention = self.softmax(energy) 
        # B X (N) X (N) #计算了一张图中每个像素点与其它像素点的关系，
        #对于一个像素点，其与其它N个像素点的关系都可以得到
        #将N展开成WxH，可以得到，该像素点的对其它所有特征点的attention map
        
        proj_value = self.value_conv(x).view(m_batchsize,-1,width*height) # B X C X N

        #bmm中的b表示的是batch ，将attention施加到特征图上，得到最终的特征图
        out = torch.bmm(proj_value,attention.permute(0,2,1) ) 
        out = out.view(m_batchsize,C,width,height)
        out = self.gamma*out + x
        return out,attention

## 源码解释

- 来自于https://zhuanlan.zhihu.com/p/110130098

在forward函数中，定义了self-attention的具体步骤。

步骤一：

proj_query  = self.query_conv(x).view(m_batchsize,-1,width*height).permute(0,2,1)
proj_query本质上就是卷积，只不过加入了reshape的操作。首先是对输入的feature map进行query_conv卷积，输出为B×C/8×W×H；view函数是改变了输出的维度，就单张feature map而言，就是将W×H大小拉直，变为1×(W×H)大小；就batchsize大小而言，输出就是B×C/8×(W×H)；permute函数则对第二维和第三维进行倒置，输出为B×(W×H)×C/8。proj_query中的第i行表示第i个像素位置上所有通道的值。




proj_key =  self.key_conv(x).view(m_batchsize,-1,width*height)
proj_key与proj_query相似，只是没有最后一步倒置，输出为B×C/8×(W×H)。proj_key中的第j行表示第j个像素位置上所有通道的值。


步骤二：

energy =  torch.bmm(proj_query,proj_key)
这一步是将batch_size中的每一对proj_query和proj_key分别进行矩阵相乘，输出为B×(W×H)×(W×H)。Energy中的第(i,j)是将proj_query中的第i行与proj_key中的第j行点乘得到。这个步骤的意义是energy中第(i,j)位置的元素是指输入特征图第j个元素对第i个元素的影响，从而实现全局上下文任意两个元素的依赖关系。




步骤三：

attention = self.softmax(energy)
这一步是将energe进行softmax归一化，是对行的归一化。归一化后每行的之和为1，对于(i,j)位置即可理解为第j位置对i位置的权重，所有的j对i位置的权重之和为1，此时得到attention_map。

proj_value = self.value_conv(x).view(m_batchsize,-1,width*height)
proj_value和proj_query与proj_key一样，只是输入为B×C×W×H，输出为B×C×(W×H)。从self-attention结构图中可以知道proj_value是与attention_map进行矩阵相乘，即下面两行代码。

out = torch.bmm(proj_value,attention.permute(0,2,1) )
out = out.view(m_batchsize,C,width,height)
在对proj_value与attention_map点乘之前，先对attention进行转置。这是由于attention中每一行的权重之和为1，是原特征图第j个位置对第i个位置的权重，将其转置之后，每一列之和为1；proj_value的每一行与attention中的每一列点乘，将权重施加于proj_value上，输出为B×C×(W×H)。

out = self.gamma*out + x
这一步是对attention之后的out进行加权，x是原始的特征图，将其叠加在原始特征图上。Gamma是经过学习得到的，初始gamma为0，输出即原始特征图，随着学习的深入，在原始特征图上增加了加权的attention，得到特征图中任意两个位置的全局依赖关系。

## attention map的可视化

- 在测试的时候，将上面代码中的attention保存下来即可可视化
- We visualize the attention maps of the last generator layer that used attention, since this layer is the closest to the output pixels and is the most straightforward to project into pixel space and interpret.
- SA GAN使用的是generator的最后一个使用attention的层来可视化。

## 疑问

向量的similarity 与注意力有什么关系。
如何理解paper中的这句话：
where sji measures the ith position’s impact on jth position.
The more similar feature representations of the two
position contributes to greater correlation between them.